In [82]:
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import os
from torchvision.datasets import VOCSegmentation
from torch.utils.data import Dataset , DataLoader
import copy
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import pickle
writer = SummaryWriter('./testlogsA')
from torch.utils.data.dataset import Dataset

In [83]:
class adversarial_pascal(Dataset):

  def __init__(self,img_transformer,target_transform ):
    with open("./Adversarial_img/DAG_A.pickle","rb") as fr:
      data = pickle.load(fr)
    self.data = data
    self.img_transform = img_transformer
    self.target_transform = target_transform

    

  def __len__(self): 
    return len(self.data) #1449

  def __getitem__(self, idx): 

    img = self.data[idx][0] # 3 256 256 np
    label = np.expand_dims(self.data[idx][1],axis=0) # 1 256 256 np
    return img,label
  
image_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5) , (0.5,0.5,0.5))
    ])

target_transform = transforms.Compose([
        transforms.ToTensor(),
        ])
        

adv_set = adversarial_pascal(image_transforms,target_transform)
  
test_loader = DataLoader(adv_set,batch_size=1,num_workers=2)



In [86]:
from UNet import UNet
device = torch.device("cuda")
model = UNet(3,21)
model = (torch.load('./UNet_with_dice.pth'))
model = model.to(device)
print(model)

import torch.nn as nn
celoss = nn.CrossEntropyLoss(ignore_index=255) 

UNet(
  (inc): double_conv(
    (conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): contraction_path(
    (contract): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): double_conv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, momentum=0

In [87]:
def make_sementic_image(img): # img is 256 256 tensor
    colors = [
    [128, 128, 128],    
    [0, 0, 255],   
    [0, 255, 0],   
    [255, 0, 0],   
    [255, 255, 0], 
    [0, 255, 255], 
    [255, 0, 255], 
    [192, 192, 192], 
    [128, 128, 128], 
    [128, 0, 0],   
    [128, 128, 0], 
    [0, 128, 0],   
    [128, 0, 128], 
    [0, 128, 128], 
    [0, 0, 128],   
    [139, 69, 19], 
    [255, 165, 0],
    [255, 192, 203],
    [255, 255, 255], 
    [255, 105, 180], 
    [240, 230, 140]  
    ]   
    #print(img.shape)
    rt = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            rt[i, j] = colors[img[i, j]]
    return torch.from_numpy(rt).permute(2,0,1)

In [89]:
total_test_loss = 0.0
total_dice_score = 0.0
total_dice_nobg= 0.0
total_ce_loss = 0.0
total_miou = 0.0
with torch.no_grad():
        for i ,data in enumerate(test_loader):
                inputs,labels = data
                inputs = inputs.to(device)
                labels=labels.squeeze(1) # change to 16 21 256 256
                labels = (labels).to(device)

                outputs = model(inputs)

                outputs_softmax = outputs.softmax(dim=1)
                celossval=celoss(outputs_softmax,labels)
                labels_no_255 = torch.where(labels >= 255, torch.zeros_like(labels), labels)
                loss = celossval
                total_ce_loss += celossval.item()
                total_test_loss+= loss.item()

                out_with_color = make_sementic_image(torch.argmax(outputs_softmax,dim=1)[0]) # voc%16==1 [1]is error
                writer.add_image('original_image',(inputs[0]*0.5+0.5),i)
                writer.add_image('model_out image',out_with_color,i)
                
print('test_loss : ',total_test_loss/len(adv_set))
print('CE loss : ',total_ce_loss/len(adv_set))


test_loss :  2.689574536101906
CE loss :  2.4157143287578884
test_loss :  2.689574536101906
CE loss :  2.4157143287578884
dice score :  tensor(0.7261, device='cuda:0')
mIOU :  0.5993790808700581
Total Dice score without BG :  0.09489458589164085
